# Evolution

Compute random graphs and statistics from multiple tournesol datasets of different dates to compare them

## 1. Init or update Tournesol dataset (Todo once every week)

- Download zip from https://api.tournesol.app/exports/all/
- Extract zip
	- Recommended to do so in directories names with the week date like: `tournesol_dataset_2023-12-31/`

## 2. Init or update Youtube cache (if tournesol dataset was updated)

Run script `py src/rndstats.py (-t <latest TOURNESOL_DATASET_PATH>) (-c <YTDATA_CACHE_PATH>) --fetch`

## 3. Check and update Notebook variables

See bloc [2] below

In [ ]:
# Imports
import os
import re
import sys
import math
import html
import glob
import colorsys
import numpy as np
import matplotlib.dates as mtdt
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime
from IPython.display import SVG

# Ensure notebook is running from Tournesol-Stats dir
_pwd = os.path.realpath('.').split(os.sep)
if 'src' in _pwd:
	while _pwd[-1] != 'src':
		_pwd.pop()
	_pwd.pop() # Go up from src dir to Tournesol-Stats
	os.chdir(os.sep.join(_pwd))
print(os.path.realpath('.'))

# Local project requirements
sys.path.append('src/py')
from dao.youtube_api import YoutubeAPI
from model.tournesol_dataset.users import extractAllTournesolUsers, TournesolUser
from model.tournesol_dataset.comparisons import ComparisonFile, ComparisonLine
from model.tournesol_dataset.collectivecriteriascores import CollectiveCriteriaScoresFile
from model.tournesol_dataset.individualcriteriascores import IndividualCriteriaScoresFile


In [ ]:
# Notebook Variables
TOURNESOL_DATASETS_PARENT_PATH='./data/' # Expecting data files located into: <TOURNESOL_DATASETS_PARENT_PATH>/**/<TOURNESOL_DATASET_PATH_FORMAT>
TOURNESOL_DATASET_FILE_FORMAT=r'.*[/\\]tournesol_dataset_([0-9]{4}-[0-9]{2}-[0-9]{2}).zip$' # SHOULD END in "yyyy-mm-dd"
YTDATA_CACHE_PATH='./data/YTData_cache.json.gz' ## TODO: DEPRECATED - Use youtubeAPI instead

In [ ]:
# Constants
TOURNESOL_DATASET_PATHS: dict[str, str]={} # date: path
for filename in glob.iglob(TOURNESOL_DATASETS_PARENT_PATH + '**/*.zip', recursive=True):
	match = re.match(TOURNESOL_DATASET_FILE_FORMAT, filename)
	if match:
		TOURNESOL_DATASET_PATHS[match.group(1)] = os.path.abspath(filename)

DATE_MAP = {s: datetime.strptime(s, '%Y-%m-%d').date() for s in TOURNESOL_DATASET_PATHS}
SORTED_DATES = sorted((k,v) for (k,v) in DATE_MAP.items())
LAST = SORTED_DATES[-1][0]
print('Dataset snapshots found:', len(TOURNESOL_DATASET_PATHS), f"(from {min(TOURNESOL_DATASET_PATHS.keys())} to {max(TOURNESOL_DATASET_PATHS.keys())})")

# Data files
USERS:set[TournesolUser] = extractAllTournesolUsers(TOURNESOL_DATASET_PATHS[LAST])
COMPARISONS:dict[str,ComparisonFile] = dict()
COLLECTIVE_SCORES:dict[str,CollectiveCriteriaScoresFile] = dict()
INDIVIDUAL_SCORES:dict[str,IndividualCriteriaScoresFile] = dict()
for date in TOURNESOL_DATASET_PATHS:
	COMPARISONS[date] = ComparisonFile(TOURNESOL_DATASET_PATHS[date])
	COLLECTIVE_SCORES[date] = CollectiveCriteriaScoresFile(TOURNESOL_DATASET_PATHS[date])
	INDIVIDUAL_SCORES[date] = IndividualCriteriaScoresFile(TOURNESOL_DATASET_PATHS[date])

# Youtube Data
YTDATA = YoutubeAPI()
try:
	YTDATA.load(YTDATA_CACHE_PATH)
except FileNotFoundError as e:
	pass

-----

# Users impact

In [ ]:
# Users ranked by how much their scores changed between last 2 weeks

print("Score difference from", SORTED_DATES[-2][0], "to", SORTED_DATES[-1][0])

week1_scores = INDIVIDUAL_SCORES[SORTED_DATES[-2][0]].get_scores(criterion='largely_recommended') # [user][vid][crit] = (score, unc.)
vids = {vid for v in week1_scores.values() for vid in v}
week2_scores = INDIVIDUAL_SCORES[SORTED_DATES[-1][0]].get_scores(users=week1_scores.keys(), vids=vids) # [user][vid][crit] = (score, unc.)

users_weekscores: dict[str,list[float]] = dict() # users_weekscores[user] = [upd. score]
for usr in week1_scores:
	if len(week1_scores[usr]) < 10:
		continue
	user_weekscores = list()
	for vid in week1_scores[usr]:
		for crit in week1_scores[usr][vid]:
			if usr in week2_scores and vid in week2_scores[usr] and crit in week2_scores[usr][vid]:
				user_weekscores.append(abs(week2_scores[usr][vid][crit][0]-week1_scores[usr][vid][crit][0]))
	users_weekscores[usr] = user_weekscores

sorted_users = sorted(users_weekscores.keys(), key=lambda usr: sum(users_weekscores[usr]), reverse=True)

print('  user             sum     avg  median    max')
print('---------------  -----  ------  ------  -----')
for user in sorted_users[:10]:
	print(f"{user[:15]:<15}  {sum(users_weekscores[user]):5.0f}  {np.average(users_weekscores[user]):6.2f}  {np.quantile(users_weekscores[user],0.5):6.2f}  {max(users_weekscores[user]):5.1f}")


In [ ]:
# Users ranked by how much the score of their comparisons changed between last 2 weeks

def users_impact(datemin:str, datemax:str):
	MIN_TRUST_SCORE = 0.01 # Add this value to all trust scores to account for users with 0 trust score (to be improved by retroengineering trust algorithm, see https://arxiv.org/pdf/2211.01179.pdf)

	# Users to consider
	users_last_week_comparisons: dict[str,set[str]] = dict()# [user] = {vid, ...}
	def extractVids(line:ComparisonLine):
		if line.criterion == 'largely_recommended' and line.date >= datemin and line.date < datemax:
			user_set = users_last_week_comparisons.setdefault(line.user,set())
			user_set.add(line.vid1)
			user_set.add(line.vid2)
	COMPARISONS[LAST].foreach(extractVids)
	users = set(users_last_week_comparisons.keys())

	# Users trust score
	trust_scores = {u.public_username: u.trust_score + MIN_TRUST_SCORE for u in USERS if u.public_username in users}

	# Individual scores differences
	week1_scores = INDIVIDUAL_SCORES[datemin].get_scores(users=users, criterion='largely_recommended') # [user][vid][crit] = (score, unc.)
	week2_scores = INDIVIDUAL_SCORES[datemax].get_scores(users=users, criterion='largely_recommended') # [user][vid][crit] = (score, unc.)
	weighted_indiv_diff = dict()
	for u in users:
		if not (u in week1_scores and u in week2_scores and u in trust_scores):
			continue
		weighted_diff = {
			v: (week2_scores[u][v]['largely_recommended'][0]-week1_scores[u][v]['largely_recommended'][0])*trust_scores[u]
			for v in set(week1_scores[u].keys()).intersection(week2_scores[u].keys()) 
		}
		weighted_indiv_diff[u] = weighted_diff

	# Collective scores differences
	week1_scores = COLLECTIVE_SCORES[datemin].get_scores(criterion='largely_recommended') # [vid][crit] = (score, unc.)
	week2_scores = COLLECTIVE_SCORES[datemax].get_scores(criterion='largely_recommended') # [vid][crit] = (score, unc.)
	coll_diff = {v: week2_scores[v]['largely_recommended'][0]-week1_scores[v]['largely_recommended'][0] for v in week1_scores if v in week2_scores}

	# Decompose collective scores differences from individual scores
	impact_from_users:dict[str,dict[str,float]] = dict() # [user][vid] = collectiveimpact
	impact_on_videos:dict[str,dict[str,float]] = dict() # [vid][user] = collectiveimpact (such as sum of [vid].values() == coll_diff[vid])
	for v in coll_diff:
		if coll_diff[v] < 0.1: # Ignore low changes
			continue
		total_impact = 0
		users_impact = {} # [user] = individualimpact (value between -220 and +220, most of the time will be around 0)
		for u in weighted_indiv_diff:
			if not v in weighted_indiv_diff[u]:
				continue
			users_impact[u] = weighted_indiv_diff[u][v]
			total_impact += weighted_indiv_diff[u][v]
		
		# Controls to prevent errors
		if abs(total_impact) < 0.01: # No (or almost no) impact found, ignore
			# print(f"Failed to compute impact for {v}: Collective score change is {coll_diff[v]:.2f} but cumulated individual change is negligeable ({total_impact})")
			continue
		if np.sign(total_impact) != np.sign(coll_diff[v]):
			# print(f"Failed to compute impact for {v}: Collective score change is {coll_diff[v]:.2f} but cumulated individual change is {total_impact:.2f}")
			continue

		# Compute every user impact (dividing total impact by collective score change)
		impact_on_videos[v] = dict()
		for u in users_impact:
			coll_impact = users_impact[u]/total_impact * coll_diff[v]
			impact_on_videos[v][u] = coll_impact
			impact_from_users.setdefault(u, dict())[v] = coll_impact
	


	# Sum users absolute impact, and print top 10
	def print_user_impact(u, maximpacted=5):
		sorted_impact = [v for v in sorted(impact_from_users[u], key=lambda v: abs(impact_from_users[u][v]), reverse=True) if abs(impact_from_users[u][v] >= 0.05)]
		positiveimpact = sum(s for s in impact_from_users[u].values() if s > 0)
		negativeimpact = -sum(s for s in impact_from_users[u].values() if s < 0)
		print(
			f"- {u} impacted the score of {len(sorted_impact)} video{'s' if len(sorted_impact) != 1 else ''}",
			(f"(↗️{positiveimpact:.1f} ↘️{negativeimpact:.1f})" if (positiveimpact > 0.05 and negativeimpact > 0.05) else (f"(↗️{positiveimpact:.1f})" if positiveimpact > 0.05 else f"(↘️{negativeimpact:.1f})"))
		)
		if maximpacted > 0:
			print('    ', ', '.join(f"{sorted_impact[i]} ({impact_from_users[u][sorted_impact[i]]:+.1f})" for i in range(0, min(maximpacted, len(sorted_impact)))), '...' if len(sorted_impact) > maximpacted else '')
	
	def print_video_impact(v, maximpacted=5):
		sorted_impact = [u for u in sorted(impact_on_videos[v], key=lambda u: abs(impact_on_videos[v][u]), reverse=True) if abs(impact_on_videos[v][u]) >= 0.05]
		positiveimpact = sum(s for s in impact_on_videos[v].values() if s > 0)
		negativeimpact = -sum(s for s in impact_on_videos[v].values() if s < 0)
		print(
			f"- {v}: Score {week1_scores[v]['largely_recommended'][0]:.1f} => {week2_scores[v]['largely_recommended'][0]:.1f} updated by {len(sorted_impact)} contributor{'s' if len(sorted_impact) != 1 else ''}",
			(f"(↗️{positiveimpact:.1f} ↘️{negativeimpact:.1f})" if (positiveimpact > 0.05 and negativeimpact > 0.05) else (f"(↗️{positiveimpact:.1f})" if positiveimpact > 0.05 else f"(↘️{negativeimpact:.1f})"))
		)
		if maximpacted > 0:
			print('    ', ', '.join(f"{sorted_impact[i]} ({impact_on_videos[v][sorted_impact[i]]:+.1f})" for i in range(0, min(maximpacted, len(sorted_impact)))), '...' if len(sorted_impact) > maximpacted else '')

	print()
	print('User impacts:')
	cumulated_user_impact = {u: np.sum(np.abs(list(impact_from_users[u].values()))) for u in impact_from_users}
	for u in sorted(cumulated_user_impact, key=cumulated_user_impact.get, reverse=True)[:20]:
		print_user_impact(u, 0)
		if u == 'NatNgs':
			break
	print()
	print('Most impacted recommended videos:')
	cumulated_video_impact = {v: np.sum(np.abs(list(impact_on_videos[v].values())))
		for v in impact_on_videos
		if week1_scores[v]['largely_recommended'][0] >= 20
		or week2_scores[v]['largely_recommended'][0] >= 20
	}
	for v in sorted(cumulated_video_impact, key=cumulated_video_impact.get, reverse=True)[:20]:
		print_video_impact(v, 0)

	print()

users_impact(SORTED_DATES[-2][0], SORTED_DATES[-1][0])



-----
# Misc

In [ ]:
## Top & growing

prev_scores = None
for d,_ in SORTED_DATES:
	scores = COLLECTIVE_SCORES[d].get_scores(criterion='largely_recommended') # (score, uncertainty) = allscores[video]['largely_recommended']
	if not prev_scores:
		prev_scores = scores
		continue

	for vid in list(scores.keys()):
		if not vid in prev_scores:
			prev_scores[vid] = scores[vid]
	
	growth:list[tuple[float,str]] = list(map(lambda vid: (abs(scores[vid]['largely_recommended'][0] - prev_scores[vid]['largely_recommended'][0]), vid), scores.keys())) ## + & -
	#growth:list[tuple[float,str]] = list(map(lambda vid: (scores[vid]['largely_recommended'][0] - prev_scores[vid]['largely_recommended'][0], vid), scores.keys())) ## Growing only
	#growth:list[tuple[float,str]] = list(map(lambda vid: (prev_scores[vid]['largely_recommended'][0] - scores[vid]['largely_recommended'][0], vid), scores.keys())) ## Falling only
	growth.sort(reverse=True)

	print('Week ' + d)
	for i in range(10):
		vid = growth[i][1]
		prev = prev_scores[vid]['largely_recommended'][0]
		news = scores[vid]['largely_recommended'][0]
		if news > prev:
			print(f"{i+1:2d}: ({prev:+3.0f} => {news:+3.0f} ↗{growth[i][0]:4.1f})", YTDATA.videos.get(vid, vid))
		else:
			print(f"{i+1:2d}: ({prev:+3.0f} => {news:+3.0f} ↘{growth[i][0]:4.1f})", YTDATA.videos.get(vid, vid))
	print()
	prev_scores = scores


In [ ]:
USER = 'NatNgs'

## Video score over time (part 1)
# Plot lines: one line per video (opacity: number of users having compared the video - 1=0%..10+=100%)
# x axis: Number of others videos it has been compared with
# y axis: Score (-100 to +100)

# Fetch scores
vid_date_vals:dict[str,dict[str,tuple[int,int,float]]] = dict() # vid_date_vals[vid][date] = (users,vid_cmps,score)
max_cmps = 0
max_usrs = 0

def fetch_collective_scores():
	vid_date_data:dict[str,dict[str,tuple[set[str],set[str]]]] = dict() # vid_date_data[vid][date] = (users,vid_cmps)
	# Fetch users & comparisons per date
	def fetch_user_data(line: ComparisonLine):
		if line.criterion != 'largely_recommended':
			return
		vid1_d = vid_date_data.setdefault(line.vid1, dict()).setdefault(line.date, (set(),set()))
		vid1_d[0].add(line.user)
		vid1_d[1].add(line.vid2)
		vid2_d = vid_date_data.setdefault(line.vid2, dict()).setdefault(line.date, (set(),set()))
		vid2_d[0].add(line.user)
		vid2_d[1].add(line.vid1)
	print('Fetching Comparisons', end='')
	for comparisons in COMPARISONS.values():
		print('.', end='', flush=True)
		comparisons.foreach(fetch_user_data)
	print()

	print('Fetching scores', end='')
	for (date,collectivescore) in COLLECTIVE_SCORES.items():
		print('.', end='', flush=True)
		scores = collectivescore.get_scores(criterion='largely_recommended', vids=vid_date_data.keys()) # (score,uncertainty) = out[vid]['largely_recommended']
		for vid in scores:
			usrs = set()
			cmps = 0
			for d in vid_date_data[vid]:
				if d <= date:
					usrs.update(vid_date_data[vid][d][0])
					cmps += len(vid_date_data[vid][d][1])
			s = scores[vid]['largely_recommended'][0]

			# Ignore if less than 3 users
			if len(usrs) < 3: continue
			if len(usrs) > max_usrs: max_usrs = len(usrs)
			if cmps > max_cmps: max_cmps = cmps

			vid_date_vals.setdefault(vid,dict())[date] = (len(usrs),cmps,s)
	print()
	return (max_usrs, max_cmps)

def fetch_individual_scores():

	vid_date_data:dict[str,dict[str,set[str]]] = dict() # vid_date_data[vid][date] = vid_cmps
	# Fetch users & comparisons per date
	def fetch_user_data(line: ComparisonLine):
		if line.criterion != 'largely_recommended' or line.user != USER:
			return
		vid_date_data.setdefault(line.vid1, dict()).setdefault(line.date, set()).add(line.vid2)
		vid_date_data.setdefault(line.vid2, dict()).setdefault(line.date, set()).add(line.vid1)
	print(f'Fetching {USER} Comparisons', end='')
	for comparisons in COMPARISONS.values():
		print('.', end='', flush=True)
		comparisons.foreach(fetch_user_data)
	print()

	max_cmps = 0
	print(f'Fetching {USER} scores', end='')
	for (date,individualscores) in INDIVIDUAL_SCORES.items():
		print('.', end='', flush=True)
		scores = individualscores.get_scores(criterion='largely_recommended', vids=vid_date_data.keys(), users=[USER])[USER] # (score,uncertainty) = out[vid]['largely_recommended']
		for vid in scores:
			cmps = 0
			for d in vid_date_data[vid]:
				if d <= date:
					cmps += len(vid_date_data[vid][d])
			s = scores[vid]['largely_recommended'][0]

			if cmps > max_cmps: max_cmps = cmps

			vid_date_vals.setdefault(vid,dict())[date] = (1,cmps,s)
	print()
	return (1,max_cmps)

(max_usrs,max_cmps) = fetch_individual_scores() # fetch_collective_scores()

In [ ]:
## Video score over time (part 2)
print('Plotting...', end='\r', flush=True)

# Prepare plot
fig, ax = plt.subplots()
fig.set_size_inches(8, 6)
# ax.set_ylabel('Collective largely_recommended score')
ax.set_ylabel(f"{USER} largely_recommended score")
ax.set_xlabel('Date')

ax.grid(visible=True, which='major', axis='both', color='gray')
ax.grid(visible=True, which='minor', axis='both', color='#eee')

ax.set_ylim([-100, 100])
ax.yaxis.set_ticks([-75,-50,-25,0,25,50,75])

ax.set_xlim(xmin=SORTED_DATES[0][1], xmax=SORTED_DATES[-1][1])
ax.xaxis.set_ticks([SORTED_DATES[i][1] for i in range(len(SORTED_DATES)-1, 0, -4) if SORTED_DATES[i][0] >= '2023-08-21'])
ax.xaxis.set_ticks([s[1] for s in SORTED_DATES if s[0] >= '2023-08-21'], minor=True)
ax.xaxis.set_major_formatter(mtdt.DateFormatter('%y-%m-%d'))
#ax.set_xlim([0, max_usrs+1])
# ax.set_xscale('log')
# ax.set_xlim([1, 10**math.ceil(math.log(max_usrs,10))])
# ax.xaxis.set_major_formatter(ScalarFormatter())

# Plot lines
n=0
for (n,vid) in enumerate(vid_date_vals):
	# vid_date_vals[vid][date] = (users,vid_cmps,score)
	xx = []
	yy = []
	oo = 0
	for d in sorted(vid_date_vals[vid].keys()):
		data = vid_date_vals[vid][d] #(users,vid_cmps,score)
		if d < '2023-08-21': continue
		xx.append(DATE_MAP[d])
		yy.append(data[2])
		oo = data[0]
	oo = math.sqrt(oo) / math.sqrt(max_usrs)

	#if max(yy)-min(yy) >= 30:
	print(f"Plotting {n+1}/{len(vid_date_vals)}...   ", end='\r', flush=True)
	
	rgbcolor = colorsys.hsv_to_rgb(0 if yy[-1] < yy[0] else 0.3, 0.8, min(1,(max(yy)-min(yy))/25))
	hexcolor = "".join("%02X" % round(i*255) for i in rgbcolor)
	ax.plot(xx, yy, linewidth=1, color="#" + hexcolor + hexcolor[-2:]) # color='#0261'
		#if xx[-1] > 18:
		#ax.text(xx[-1], yy[-1], s=YTDATA.videos.get(vid,vid), horizontalalignment='left', verticalalignment='center', fontsize=6)
print()